In [2]:
#ignore warnings for clear printing
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression, LinearRegression, MultiTaskLasso,BayesianRidge, RidgeClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR, SVC
import seaborn as sns # More snazzy plotting library
import itertools
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import  KNeighborsRegressor,KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessRegressor,GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor
from sklearn.svm import LinearSVR, LinearSVC
##test results 
#  1.looks the absolute year number gives a good colleration for prediction. 
#  2. standarlistaion improves prediction by 3%
#  3. removing aspect ratio improves prediction by 2%

#file_path =  "../dataset/movie_metadata_cleaned_tfidf_num_only_min.csv"
file_path =  "../dataset/movie_metadata_cleaned_categ_num_only.csv"
#file_path = "../dataset/movie_metadata_cleaned_no_vector_num_only.csv"

dta = pd.read_csv(file_path)
dta_clean = dta
#remove the null values, that is fill NaN with there - FIXME: Rihards, naive implementation
dta_clean = dta_clean.fillna(value=0, axis=1)
dta_clean = dta_clean.dropna()
dta_clean = dta_clean.drop('Unnamed: 0', axis=1)
#drop wordwide grosses with zero
#dta_clean = dta_clean[dta_clean['worldwide_gross'] != 0]

dta_clean.describe()

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,blockbuster_month,cast_total_facebook_likes,director_facebook_likes,dump_month,duration,imdb_score,movie_facebook_likes,...,raiting_pg,raiting_pg13,raiting_r,raiting_tv14,raiting_tvg,raiting_tvma,raiting_tvpg,raiting_tvy7,raiting_unrated,raiting_x
count,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,...,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000,4812.000000
mean,6801.628221,1720.181629,662.147548,0.368869,10068.043225,742.647756,0.219659,106.889443,6.417249,8067.969659,...,0.141521,0.297589,0.427265,0.003741,0.001455,0.002702,0.001870,0.000208,0.011222,0.000416
std,15654.706536,4105.115266,1665.732147,0.482549,18756.607185,2934.558160,0.414059,21.994601,1.105504,20009.801260,...,0.348595,0.457245,0.494733,0.061053,0.038117,0.051912,0.043211,0.014416,0.105349,0.020385
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.600000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,644.750000,310.000000,154.000000,0.000000,1563.250000,8.000000,0.000000,93.000000,5.800000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1000.000000,628.000000,389.000000,0.000000,3258.000000,53.000000,0.000000,103.000000,6.500000,192.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,12000.000000,942.000000,650.000000,1.000000,14618.750000,212.000000,0.000000,118.000000,7.200000,7000.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,713000.000000,137000.000000,23000.000000,1.000000,729779.000000,33000.000000,1.000000,227.000000,9.600000,353000.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
model.n_iter_

1000

In [3]:
#clasify the data for the logistic regression
def label_gross (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 10000000)) : return 2
    elif ((gross >= 10000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 200000000)) : return 4
    elif (gross >= 200000000) : return 5
    
#y = dta_clean.worldwide_gross.apply (lambda gross: label_gross (gross))
y = dta_clean['worldwide_gross']
X = dta_clean.drop('worldwide_gross', axis=1)

X_col = X.columns
if False:
    nomalizer_scaler = preprocessing.Normalizer().fit(X)
    X = nomalizer_scaler.transform(X)
if False:
    standard_scaler = preprocessing.StandardScaler().fit(X)
    X = standard_scaler.transform(X)
        
#pd.DataFrame(X, columns=X_col).describe()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
#model = SVR(kernel='rbf', verbose=2)

#model = LinearRegression()
#model = LogisticRegression()

#model = DecisionTreeRegressor()
#model = DecisionTreeClassifier()

#model = BayesianRidge()
#model = RidgeClassifier()

#model = KNeighborsRegressor()
#model = KNeighborsClassifier()

model = GradientBoostingRegressor()
#from sklearn.linear_model import   ARDRegression
#model = GradientBoostingClassifier()

#model = AdaBoostRegressor()
#model = AdaBoostClassifier()

#model = LinearSVR()
#model = LinearSVC()

#model = SVR(kernel='rbf')
#model = SVC(kernel='rbf')

#model = MLPRegressor()
#model = MLPClassifier(solver='lbfgs',  max_iter=1000)

#model.fit(X_train, y_train)
# check the accuracy on the training set
#model.predict(X_test)
#LR_1.score(X_1, y_1)
#model.score(X_test, y_test)
#score_re = {}
#scorers = ['accuracy', 'adjusted_rand_score', 'average_precision', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc']
#for scoring in scorers:
#    try:
#        print(scoring)
#        score_re[scoring] = cross_val_score(model, X, y, scoring=scoring)
#    except (ValueError, AttributeError): 
#        continue
cross_val_score(model, X, y, scoring="r2") 

array([  -0.7019762 ,  -14.73256154, -463.05439037])

In [63]:
print(score_re)

{'r2': array([  -0.70100238,  -14.80351263, -458.0504105 ]), 'neg_mean_absolute_error': array([ -1.90603929e+08,  -4.15342620e+07,  -4.16701420e+07]), 'neg_median_absolute_error': array([ -1.13706829e+08,  -2.85080300e+07,  -2.52039684e+07]), 'adjusted_rand_score': array([  7.64699746e-01,   4.60742861e-01,   5.90487049e-04]), 'neg_mean_squared_error': array([ -8.47574903e+16,  -3.66577884e+15,  -4.31268330e+15])}
